# Genetic Test Expression to OMOP

Genetic test expression is mapped to prcedure occurrence, following: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=728670853#gid=728670853

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
df_genetic_test_expression_IDEA4RC = pd.read_csv("./IDEA4RC-data/geneticTestExpressionIDEA4RC.csv")
df_genetic_test_expression_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Genetic Test Expression to Procedure Occurrence

We need a visit occurrence, however, we do not have any for these procedures. What should I do with this then?

In [ ]:
sql = """
    INSERT INTO omopcdm.condition_occurrence (person_id, procedure_concept_id, procedure_date, procedure_type_concept_id)
    VALUES (%s, %s, %s, %s)
"""
columns={
    "gene_expression_analysis_performed": "date_of_gene_expression",
    "gene_mutation_analysis_performed":"date_of_gene_mutation",
    "tests_for_chromosome_translocations_performed":"date_of_translocation",
    "ngs_performed":"date_of_ngs",
    "pcr_test_performed":"date_of_pcr",
    "immunohistochemistry_performed":"date_of_immunohistochemistry",
    "ct_dna_performed":"date_of_ct_dna"
}
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""
vocabs={
    "gene_expression_analysis_performed": 44808037,
    "gene_mutation_analysis_performed": 4038339,
    "tests_for_chromosome_translocations_performed": 4039364,
    "ngs_performed": 40485067,
    "pcr_test_performed": 40482792,
    "immunohistochemistry_performed": 4035726,
    "ct_dna_performed": 19387013,
}

procedure_type_concept_id=32879

df_tables=df_genetic_test_expression_IDEA4RC
for idx, row in df_tables.iterrows():
    for column in columns.keys():
        if row[column] == 1 or row[column]:
            curIDEA.execute(query, (row['episode_event']))
            procedure_concept_id=vocabs.get(column)
            res=curIDEA.fetchone()
            person_id=res
            procedure_date_var=columns.get(column)
            procedure_date=row[procedure_date_var]
            cur.execute(sql, (person_id, procedure_concept_id, procedure_date, procedure_type_concept_id))
        